<a href="https://colab.research.google.com/github/nekoniii3/Colabo_Samples/blob/main/035_GPT4_FineTuning/OpenAI_%E7%94%BB%E5%83%8FFine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenAIインストール

In [ ]:
!pip install openai -q

# APIキーのセットなど各種下準備

In [2]:
import os
import base64
from openai import OpenAI

from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_KEY")
# os.environ["OPENAI_API_KEY"] = "Enter "OpenAI API Key"    # 直接入力でも構いません

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# 画像ファイルアップ・解凍・データ取得

In [ ]:
# 画像を圧縮したZIPファイルをアップして下さい
zip_name = "FT_DATA.zip"

!unzip $zip_name

In [5]:
data_dir = "/content/FT_DATA/"

# ファイル一覧を取得
files = os.listdir(data_dir)

files.sort()    # 並び替え

In [6]:
img_list = []

for file in files:

    img_list.append([file, encode_image(data_dir + file)])

# 学習用データ（JSONL）出力

In [7]:
output = ""

filename = "pl_data_01.jsonl"

system_instruction = "あなたは優秀なゲーム解説者です。"
user_question = "この画像のキャラクターは何というゲームのキャラクターですか？"

for img_data in img_list:

    chara_name = img_data[0].split(".")[0]

    answer = f"これは日本のゲーム「パルワールド」のキャラクターの「{chara_name}」です。"

    output += '{"messages": [{"role": "system", "content": "' + system_instruction + '"}, \
      {"role": "user", "content": "'+ user_question +'"}, \
      {"role": "user", "content": [ \
          { \
            "type": "image_url", \
            "image_url": { \
                "url": "data:image/jpeg;base64,' + img_data[1] + '" \
            } \
          } \
        ] \
       }, \
       {"role": "assistant", "content": "' + answer + '"} \
    ]}\n'

In [8]:
# ファイルとして出力
with open(filename, "w") as file:
    file.write(output)

# 学習用データ（JSONL）のアップロード

In [9]:
client = OpenAI()

# 学習データのアップロード
file_create = client.files.create(
    file=open(filename, "rb"),
    purpose="fine-tune"
)

print(file_create.id)

file-ryKToaRQEGdSctiu10uHbz6T


# ファインチューニング実行

In [10]:
file_id = file_create.id
# file_id = "…"   # 直接指定しても構いません

GPT_MODEL = "gpt-4o-2024-08-06"

ftjob = client.fine_tuning.jobs.create(
  training_file=file_id,
  model=GPT_MODEL
)

# ファインチューニング状況確認
**※ダッシュボードでも確認できます<br>**
https://platform.openai.com/finetune

In [14]:
# ファインチューニング状況確認
ftjob = client.fine_tuning.jobs.retrieve(ftjob.id)

print(ftjob.status)

succeeded


# 動作確認
**※プレイグラウンドでも確認できます<br>**
https://platform.openai.com/playground

In [18]:
image_path = "/content/FT_DATA/ツッパニャン.jpg"

base64_image = encode_image(image_path)

In [ ]:
model_name = ftjob.fine_tuned_model
# model_name = "ft:gpt-…"   # モデル名指定

response = client.chat.completions.create(
    model=model_name,
    messages=[
      {
        "role": "user",
        "content": [
          {"type": "text", "text": "あなたはゲーム解説者です。この画像のキャラクターは何のキャラクターでしょうか？"},
          {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{base64_image}"
            },
          },
        ],
      }
    ],
)

print(response.choices[0].message.content)